## Objetivo

Me interesa saber la distribución de los costos de productos adquiridos por la secretaría de salud. Esto es, ¿Cuánto de más o cuánto de menos se ha estado pagando? ¿Podemos detectar corrupción? ¿Comportamiento por distribuidores?

## Obtener la data

No logre encontrar los datos abiertos que alimentan: http://compras.imss.gob.mx/?P=search_alt Voy a proceder a scrappearlo.

Alto, están las compras consolidadas: Según HHealth (Gracias!) no son todas las compras. pero está chido para hacerme de un diccionario.
http://www.imss.gob.mx/compraconsolidada2016

In [1]:
import requests as r
import os
import json
from bs4 import BeautifulSoup

In [2]:
def products(field,page):
    result = os.popen("curl 'http://buscador.compras.imss.gob.mx/index.php' -H 'Pragma: no-cache' -H 'Origin: http://buscador.compras.imss.gob.mx' -H 'Accept-Encoding: gzip, deflate' -H 'Accept-Language: es-ES,es;q=0.8,en;q=0.6' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36' -H 'Content-Type: application/x-www-form-urlencoded' -H 'Accept: */*' -H 'Cache-Control: no-cache' -H 'X-Requested-With: WAJAF::Ajax - WebAbility(r) v5' -H 'Cookie: __utma=37030712.222899114.1488655887.1488656746.1488656746.1; __utmb=37030712.1.10.1488656746; __utmc=37030712; __utmz=37030712.1488656746.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __asc=9819a67f15a9ace2b0599c2b83a; __auc=9819a67f15a9ace2b0599c2b83a; _ga=GA1.3.222899114.1488655887' -H 'Connection: keep-alive' -H 'Referer: http://buscador.compras.imss.gob.mx/wrap/index.html' --data '&type=compras&message=X&filtered=1&descripcion="+field+"&proveedor=&numcompra=&delegacion=values%3D&fecha=min%3D%3Bmax%3D&procedimiento=values%3D&exact=false&numperpage=20&page=" + str(page)+"&order=fecha%20desc' --compressed").read()
    return(json.loads(result))

In [17]:
first_page  =products(" ",1)

In [8]:
first_page["result"].keys()

dict_keys(['id', 'exact', 'nquery', 'numperpage', 'quantity', 'page', 'data', 'time', 'near'])

In [18]:
import math

In [9]:
int(first_page["result"]["quantity"])

2138520

In [21]:
int(int(first_page["result"]["quantity"])/20)

106926

In [23]:
first_page["result"]["data"]

[{'clave': '12174280',
  'delegacion': None,
  'descripcion': 'PEDMAN004',
  'fecha': '',
  'monto': '0.00',
  'numcompra': None,
  'procedimiento': None,
  'proveedor': None},
 {'clave': '12174279',
  'delegacion': None,
  'descripcion': 'E252017',
  'fecha': '',
  'monto': '0.00',
  'numcompra': None,
  'procedimiento': None,
  'proveedor': None},
 {'clave': '12174278',
  'delegacion': None,
  'descripcion': 'IA019GYR023E32',
  'fecha': '',
  'monto': '0.00',
  'numcompra': None,
  'procedimiento': None,
  'proveedor': None},
 {'clave': '12174277',
  'delegacion': None,
  'descripcion': 'IA019GYR023E32',
  'fecha': '',
  'monto': '0.00',
  'numcompra': None,
  'procedimiento': None,
  'proveedor': None},
 {'clave': '12168168',
  'delegacion': None,
  'descripcion': 'D7P0057',
  'fecha': '',
  'monto': '0.00',
  'numcompra': None,
  'procedimiento': None,
  'proveedor': None},
 {'clave': '12168167',
  'delegacion': None,
  'descripcion': 'D7P0059',
  'fecha': '',
  'monto': '0.00',
  

In [11]:
def transaction(transaction_number):
    transaction_page = "http://compras.imss.gob.mx/?P=imsscomprofich&f="+ str(transaction_number)
    content = r.get(transaction_page)
    soup = BeautifulSoup(content.content)
    local_data = {}
    for span in soup.findAll("span", {"class":"txtdesccaja"}):
        split_span = span.text.replace("\n","").strip().split(":")
        name = split_span[0]
        content = " ".join(split_span[1:])
        local_data[name]=content
    return(local_data)

In [12]:
transaction(12086888)

/usr/local/lib/python3.4/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


{'# Factura': ' D17159',
 'COMERCIAL HOSPITALARIA S.A DE C.V.': '',
 'Cantidad recibida': ' 30',
 'Cantidad solicitada': ' 30',
 'Delegación del IMSS': ' Delegación Sonora',
 'Descripción': ' TIRA REACTIVA PARA DETERMINACION SEMICUANTITATIVA DE GLUCOSA EN SANGRE CON   LIMITES DE DETECCION QUE VAN DE 20 A 800 MG/DL. FRASCO CON 50 TIRAS. TA.',
 'Estado de la República': ' Sonora',
 'Fecha factura': ' 17/02/17',
 'IVA': ' 16',
 'No. Procedimiento': ' 230',
 'Precio': ' 294.00',
 'Procedimiento de compra': 'Compra emergente en unidades de servicio',
 'RFC': ' CHO011105F23 [Ver ficha del proveedor]',
 'Subprocedimiento de compra': ' N/A',
 'Unidad': ' TRA'}

In [13]:
transaction(12086887)

{'# Factura': ' D17094',
 'COMERCIAL HOSPITALARIA S.A DE C.V.': '',
 'Cantidad recibida': ' 15',
 'Cantidad solicitada': ' 15',
 'Delegación del IMSS': ' Delegación Sonora',
 'Descripción': ' RIFAMPICINA SUSPENSION ORAL CADA 5 ML CONTIENEN  RIFAMPICINA 100 MG ENVASE CON   120 ML Y DOSIFICADOR.',
 'Estado de la República': ' Sonora',
 'Fecha factura': ' 13/02/17',
 'IVA': ' No contiene datos',
 'No. Procedimiento': ' 171',
 'Precio': ' 438.90',
 'Procedimiento de compra': 'Compra emergente en unidades de servicio',
 'RFC': ' CHO011105F23 [Ver ficha del proveedor]',
 'Subprocedimiento de compra': ' N/A',
 'Unidad': ' ENV'}

In [14]:
def get_all_data():
    first_page = search_page(" ",1)
    data = first_page["result"]["data"]
    hits = int(first_page["result"]["quantity"])
    pages = floor(hits/20) + 1
    i = 1
    while i<=pages:
        try:
            page_data = search_page(" ",i)
            data = data + page_data["result"]["data"]
            i = i +1
        except:
            None
    return(data)

In [15]:
import math
math.floor(int(results["result"]["quantity"])/20)+1

NameError: name 'results' is not defined

In [ ]:
results

In [ ]:
results = search_page(" ",3)

In [ ]:
results

In [22]:
for i in range(1,5):
    print(i)

1
2
3
4


In [26]:
currentPageFile = "pagefile.txt"

In [27]:
def writePage(number):
    f = open('myfile', 'w')
    f.write('hi there\n')  # python will convert \n to os.linesep
    f.close()  # you can omit in most cases as the destructor will call it
    return

In [28]:
writePage(1)

AttributeError: 'str' object has no attribute 'write'

In [29]:
def writePage(number):
    f = open(currentPageFile, 'w')
    f.write(str(number))  # python will convert \n to os.linesep
    f.close()  # you can omit in most cases as the destructor will call it
    return "ok"


In [30]:
writePage(1)

'ok'

In [36]:
def writePage(number, quantity):
    f = open(currentPageFile, 'w')
    f.write(str(number)+ "\t" + str(quantity))  # python will convert \n to os.linesep
    f.close()  # you can omit in most cases as the destructor will call it
    return "ok"


def readPage():
    file = open(currentPageFile, "r")
    for line in file:
        number, quantity = line.split("\t")
    return int(number), int(quantity)

In [39]:
writePage(1,231)

'ok'

In [38]:
readPage()

(1, 231)